<a target="_blank" href="https://colab.research.google.com/github/wakusoftware/curso-ml-espanol/blob/master/C1%20-%20Aprendizaje%20Supervisado/C1_W1_Lab04_Descendiente_Gradiente.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup para Colab
Si estás corriendo este Notebook en Google Colab corre la celda de abajo, de lo contrario ignórala.

In [ ]:
!git clone https://github.com/wakusoftware/curso-ml-espanol.git

%cd curso-ml-espanol/C1 - Aprendizaje Supervisado/

!cp -r deeplearning.mplstyle /content/

!cp -r images /content/

!cp -r lab_utils_uni.py /content/

%cd /content/

!rm -rf curso-ml-espanol/

# Lab: Descenso de Gradiente para Regresión Lineal

<figure>
    <center> <img src="./images/C1_W1_L4_S1_Lecture_GD.jpeg"  style="width:800px;height:200px;" ></center>
</figure>

## Objetivos
En este laboratorio:
- automatizarás el proceso de optimización de $w$ y $b$ utilizando el descenso de gradiente.

## Herramientas
En este laboratorio, haremos uso de:
- NumPy, una biblioteca popular para computación científica
- Matplotlib, una biblioteca popular para graficar datos
- rutinas de graficación en el archivo lab_utils.py en el directorio local.

In [ ]:
import math, copy
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
from lab_utils_uni import plt_house_x, plt_contour_wgrad, plt_divergence, plt_gradients

# Planteamiento del problema

Utilicemos los mismos dos puntos de datos que antes: una casa de 1000 pies cuadrados se vendió por 300,000 dólares y una casa de 2000 pies cuadrados se vendió por 500,000 dólares.

| Tamaño (1000 pies cuadrados) | Precio (miles de dólares) |
| ----------------------------- | -------------------------- |
| 1                             | 300                        |
| 2                             | 500                        |

In [ ]:
# Cargar nuestro dataset
x_train = np.array([1.0, 2.0])   # características
y_train = np.array([300.0, 500.0])   # valor objetivo

<a name="toc_40291_2.0.1"></a>
### compute_cost
Esto se desarrolló en el último laboratorio. Lo necesitaremos de nuevo aquí.

In [ ]:
# Función para calcular el costo o pérdida
def compute_cost(x, y, w, b):
   
    m = x.shape[0] 
    cost = 0
    
    for i in range(m):
        f_wb = w * x[i] + b
        cost = cost + (f_wb - y[i])**2
    total_cost = 1 / (2 * m) * cost

    return total_cost

<a name="toc_40291_2.1"></a>
## Resumen del descenso de gradiente
Hasta ahora en este curso, has desarrollado un modelo lineal que predice $f_{w,b}(x^{(i)})$:
$$f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$
En la regresión lineal, utilizas datos de entrenamiento de entrada para ajustar los parámetros $w$, $b$ minimizando una medida del error entre nuestras predicciones $f_{w,b}(x^{(i)})$ y los datos reales $y^{(i)}$. La medida se llama el $costo$, $J(w,b)$. En el entrenamiento, mides el costo sobre todas nuestras muestras de entrenamiento $x^{(i)},y^{(i)}$
$$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2\tag{2}$$

En la conferencia, se describió el *descenso de gradiente* como:

$$\begin{align*} \text{repetir}&\text{ hasta la convergencia:} \; \lbrace \newline
\;  w &= w -  \alpha \frac{\partial J(w,b)}{\partial w} \tag{3}  \; \newline 
 b &= b -  \alpha \frac{\partial J(w,b)}{\partial b}  \newline \rbrace
\end{align*}$$
donde, los parámetros $w$, $b$ se actualizan simultáneamente.  
El gradiente se define como:
$$
\begin{align}
\frac{\partial J(w,b)}{\partial w}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)} \tag{4}\\
  \frac{\partial J(w,b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \tag{5}\\
\end{align}
$$

Aquí *simultáneamente* significa que calculas las derivadas parciales para todos los parámetros antes de actualizar cualquiera de los parámetros.

<a name="toc_40291_2.2"></a>
## Implementar el Descenso de Gradiente
Implementarás el algoritmo de descenso de gradiente para una característica. Necesitarás tres funciones.
- `compute_gradient` implementando las ecuaciones (4) y (5) arriba mencionadas
- `compute_cost` implementando la ecuación (2) arriba mencionada (código del laboratorio anterior)
- `gradient_descent`, utilizando compute_gradient y compute_cost

Convenciones:
- El nombramiento de variables de Python que contienen derivadas parciales sigue este patrón, $\frac{\partial J(w,b)}{\partial b}$ será `dj_db`.
- w.r.t es With Respect To (Con Respecto A), como en derivada parcial de $J(wb)$ Con Respecto A $b$.

<a name="toc_40291_2.3"></a>
### compute_gradient
<a name='ex-01'></a>
`compute_gradient` implementa (4) y (5) arriba mencionadas y devuelve $\frac{\partial J(w,b)}{\partial w}$, $\frac{\partial J(w,b)}{\partial b}$. Los comentarios incrustados describen las operaciones.

In [ ]:
def compute_gradient(x, y, w, b): 
    """
    Calcula el gradiente para la regresión lineal
    Argumentos:
      x (ndarray (m,)): Datos, m ejemplos
      y (ndarray (m,)): valores objetivo
      w,b (escalar)    : parámetros del modelo  
    Devuelve
      dj_dw (escalar): El gradiente del costo con respecto a los parámetros w
      dj_db (escalar): El gradiente del costo con respecto al parámetro b     
     """
    
    # Número de ejemplos de entrenamiento
    m = x.shape[0]    
    dj_dw = 0
    dj_db = 0
    
    for i in range(m):  
        f_wb = w * x[i] + b 
        dj_dw_i = (f_wb - y[i]) * x[i] 
        dj_db_i = f_wb - y[i] 
        dj_db += dj_db_i
        dj_dw += dj_dw_i 
    dj_dw = dj_dw / m 
    dj_db = dj_db / m 
        
    return dj_dw, dj_db

<br/>

Las clases describieron cómo el descenso de gradiente utiliza la derivada parcial del costo con respecto a un parámetro en un punto para actualizar ese parámetro.
Vamos a usar nuestra función `compute_gradient` para encontrar y graficar algunas derivadas parciales de nuestra función de costo en relación con uno de los parámetros, $w_0$.

In [ ]:
plt_gradients(x_train,y_train, compute_cost, compute_gradient)
plt.show()

Arriba, el gráfico de la izquierda muestra $\frac{\partial J(w,b)}{\partial w}$ o la pendiente de la curva de costo relativa a $w$ en tres puntos. En el lado derecho del gráfico, la derivada es positiva, mientras que en el lado izquierdo es negativa. Debido a la forma de 'cuenco', las derivadas siempre guiarán el descenso de gradiente hacia el fondo donde el gradiente es cero.

El gráfico de la izquierda tiene $b=100$ fijo. El descenso de gradiente utilizará tanto $\frac{\partial J(w,b)}{\partial w}$ como $\frac{\partial J(w,b)}{\partial b}$ para actualizar parámetros. El 'gráfico de flechas' a la derecha proporciona un medio para visualizar el gradiente de ambos parámetros. El tamaño de las flechas refleja la magnitud del gradiente en ese punto. La dirección y pendiente de la flecha refleja la proporción de $\frac{\partial J(w,b)}{\partial w}$ y $\frac{\partial J(w,b)}{\partial b}$ en ese punto.
Nota que el gradiente apunta *lejos* del mínimo. Revisa la ecuación (3) arriba. El gradiente escalado se *resta* del valor actual de $w$ o $b$. Esto mueve el parámetro en una dirección que reducirá el costo.

<a name="toc_40291_2.5"></a>
### Descenso de Gradiente
Ahora que se pueden calcular los gradientes, el descenso de gradiente, descrito en la ecuación (3) arriba, se puede implementar a continuación en `gradient_descent`. Los detalles de la implementación se describen en los comentarios. A continuación, utilizarás esta función para encontrar valores óptimos de $w$ y $b$ en los datos de entrenamiento.

In [ ]:
def gradient_descent(x, y, w_in, b_in, alpha, num_iters, cost_function, gradient_function): 
    """
    Realiza el descenso de gradiente para ajustar w,b. Actualiza w,b tomando
    num_iters pasos de gradiente con una tasa de aprendizaje alpha
    
    Argumentos:
      x (ndarray (m,))  : Datos, m ejemplos 
      y (ndarray (m,))  : valores objetivo
      w_in, b_in (escalar): valores iniciales de los parámetros del modelo  
      alpha (float):     Tasa de aprendizaje
      num_iters (int):   número de iteraciones para ejecutar el descenso de gradiente
      cost_function:     función a llamar para producir el costo
      gradient_function: función a llamar para producir el gradiente
      
    Devuelve:
      w (escalar): Valor actualizado del parámetro después de ejecutar el descenso de gradiente
      b (escalar): Valor actualizado del parámetro después de ejecutar el descenso de gradiente
      J_history (Lista): Historial de valores de costo
      p_history (lista): Historial de parámetros [w,b] 
      """
    
    w = copy.deepcopy(w_in) # evitar modificar w_in global
    # Un arreglo para almacenar el costo J y los w's en cada iteración principalmente para graficar luego
    J_history = []
    p_history = []
    b = b_in
    w = w_in
    
    for i in range(num_iters):
        # Calcular el gradiente y actualizar los parámetros usando gradient_function
        dj_dw, dj_db = gradient_function(x, y, w , b)     

        # Actualizar Parámetros usando la ecuación (3) arriba
        b = b - alpha * dj_db                            
        w = w - alpha * dj_dw                            

        # Guardar el costo J en cada iteración
        if i<100000:      # prevenir agotamiento de recursos 
            J_history.append(cost_function(x, y, w , b))
            p_history.append([w,b])
        # Imprimir el costo cada ciertos intervalos 10 veces o tantas iteraciones si son < 10
        if i% math.ceil(num_iters/10) == 0:
            print(f"Iteración {i:4}: Costo {J_history[-1]:0.2e} ",
                  f"dj_dw: {dj_dw: 0.3e}, dj_db: {dj_db: 0.3e}  ",
                  f"w: {w: 0.3e}, b:{b: 0.5e}")
 
    return w, b, J_history, p_history #devolver w y el historial de J,w para graficar


In [ ]:
# inicializar parámetros
w_init = 0
b_init = 0
# algunas configuraciones de descenso de gradiente
iteraciones = 10000
tmp_alpha = 1.0e-2
# ejecutar el descenso de gradiente
w_final, b_final, J_hist, p_hist = gradient_descent(x_train ,y_train, w_init, b_init, tmp_alpha, 
                                                    iteraciones, compute_cost, compute_gradient)
print(f"(w,b) encontrados por descenso de gradiente: ({w_final:8.4f},{b_final:8.4f})")


Tómate un momento y nota algunas características del proceso de descenso de gradiente impreso arriba.

- El costo comienza grande y disminuye rápidamente, como se describió en la diapositiva de la clase.
- Las derivadas parciales, `dj_dw`, y `dj_db` también se hacen más pequeñas, rápidamente al principio y luego más lentamente. A medida que el proceso se acerca al 'fondo del tazón', el progreso es más lento debido al menor valor de la derivada en ese punto.
- El progreso se ralentiza aunque la tasa de aprendizaje, alfa, permanece fija.

### Costo versus iteraciones del descenso de gradiente
Un gráfico del costo versus las iteraciones es una medida útil del progreso en el descenso de gradiente. El costo siempre debería disminuir en ejecuciones exitosas. El cambio en el costo es tan rápido inicialmente, que es útil graficar el descenso inicial en una escala diferente que el descenso final. En los gráficos a continuación, nota la escala del costo en los ejes y el paso de iteración.

In [ ]:
# graficar costo versus iteración
fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(12,4))
ax1.plot(J_hist[:100])
ax2.plot(1000 + np.arange(len(J_hist[1000:])), J_hist[1000:])
ax1.set_title("Costo vs. iteración (inicio)");  ax2.set_title("Costo vs. iteración (final)")
ax1.set_ylabel('Costo')            ;  ax2.set_ylabel('Costo') 
ax1.set_xlabel('paso de iteración')  ;  ax2.set_xlabel('paso de iteración') 
plt.show()


### Predicciones
Ahora que has descubierto los valores óptimos para los parámetros $w$ y $b$, ahora puedes usar el modelo para predecir valores de viviendas basados en nuestros parámetros aprendidos. Como se esperaba, los valores predichos son casi los mismos que los valores de entrenamiento para la misma vivienda. Además, el valor que no está en la predicción está en línea con el valor esperado.

In [ ]:
print(f"Predicción de casa de 1000 pies cuadrados {w_final*1.0 + b_final:0.1f} Mil dólares")
print(f"Predicción de casa de 1200 pies cuadrados {w_final*1.2 + b_final:0.1f} Mil dólares")
print(f"Predicción de casa de 2000 pies cuadrados {w_final*2.0 + b_final:0.1f} Mil dólares")

<a name="toc_40291_2.6"></a>
## Graficación
Puedes mostrar el progreso del descenso de gradiente durante su ejecución al graficar el costo sobre las iteraciones en un gráfico de contorno del costo(w,b).

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12, 6))
plt_contour_wgrad(x_train, y_train, p_hist, ax)

Arriba, el gráfico de contorno muestra el $costo(w,b)$ sobre un rango de $w$ y $b$. Los niveles de costo están representados por los anillos. Superpuesto, usando flechas rojas, está el camino del descenso de gradiente. Aquí hay algunas cosas a tener en cuenta:
- El camino hace un progreso constante (monotónico) hacia su objetivo.
- Los pasos iniciales son mucho más grandes que los pasos cerca del objetivo.

**Acercándonos**, podemos ver los pasos finales del descenso de gradiente. Nota que la distancia entre pasos se reduce a medida que el gradiente se acerca a cero.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12, 4))
plt_contour_wgrad(x_train, y_train, p_hist, ax, w_range=[180, 220, 0.5], b_range=[80, 120, 0.5],
            contours=[1,5,10,20],resolution=0.5)

<a name="toc_40291_2.7.1"></a>
### Aumento de la Tasa de Aprendizaje

<figure>
 <img align="left", src="./images/C1_W1_Lab03_alpha_too_big.jpeg"   style="width:340px;height:240px;" >
</figure>
En la conferencia, hubo una discusión relacionada con el valor adecuado de la tasa de aprendizaje, $\alpha$ en la ecuación(3). Cuanto mayor es $\alpha$, más rápido el descenso de gradiente convergerá a una solución. Pero, si es demasiado grande, el descenso de gradiente divergerá. Arriba tienes un ejemplo de una solución que converge bien.

Intentemos aumentar el valor de $\alpha$ y veamos qué sucede:

In [ ]:
# inicializar parámetros
w_init = 0
b_init = 0
# establecer alpha a un valor grande
iteraciones = 10
tmp_alpha = 8.0e-1
# ejecutar el descenso de gradiente
w_final, b_final, J_hist, p_hist = gradient_descent(x_train ,y_train, w_init, b_init, tmp_alpha, 
                                                    iteraciones, compute_cost, compute_gradient)


Arriba, $w$ y $b$ están rebotando de un lado a otro entre positivo y negativo con el valor absoluto aumentando con cada iteración. Además, en cada iteración $\frac{\partial J(w,b)}{\partial w}$ cambia de signo y el costo está aumentando en lugar de disminuir. Esto es una clara señal de que *la tasa de aprendizaje es demasiado grande* y la solución está divergiendo.
Visualicemos esto con un gráfico.

In [ ]:
plt_divergence(p_hist, J_hist,x_train, y_train)
plt.show()

Arriba, el gráfico de la izquierda muestra la progresión de $w$ durante los primeros pasos del descenso de gradiente. $w$ oscila de positivo a negativo y el costo crece rápidamente. El Descenso de Gradiente opera tanto en $w$ como en $b$ simultáneamente, por lo que se necesita el gráfico 3D de la derecha para la imagen completa.

## ¡Felicidades!
En este laboratorio:
- profundizaste en los detalles del descenso de gradiente para una variable única.
- desarrollaste una rutina para calcular el gradiente
- visualizaste qué es el gradiente
- completaste una rutina de descenso de gradiente
- utilizaste el descenso de gradiente para encontrar parámetros
- examinaste el impacto de ajustar la tasa de aprendizaje